In [1]:
import easyocr

from PIL import Image
import pytesseract

import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np
import Levenshtein as lev

from enum import IntEnum

import cv2
import os
import optuna

C:\ProgramData\Anaconda3\envs\ocr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import json

In [3]:
data_dir = r'E:\datasets\MADE\3_graduation\parthplc\archive\data\\'

train_path = data_dir + 'train.jsonl'
dev_path = data_dir + 'dev.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [4]:
train_data['path'] = train_data['img'].apply(lambda x: os.path.join(data_dir, x))
train_data

,id,img,label,text,path
0,42953,img/42953.png,0,its their character not their color that matters,E:\datasets\MADE\3_graduation\parthplc\archive...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,E:\datasets\MADE\3_graduation\parthplc\archive...
2,13894,img/13894.png,0,putting bows on your pet,E:\datasets\MADE\3_graduation\parthplc\archive...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,E:\datasets\MADE\3_graduation\parthplc\archive...
8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,E:\datasets\MADE\3_graduation\parthplc\archive...
8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,E:\datasets\MADE\3_graduation\parthplc\archive...
8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,E:\datasets\MADE\3_graduation\parthplc\archive...


In [5]:
test_data['path'] = test_data['img'].apply(lambda x: os.path.join(data_dir, x))
test_data

,id,img,label,text,path
0,8291,img/08291.png,1,white people is this a shooting range,E:\datasets\MADE\3_graduation\parthplc\archive...
1,46971,img/46971.png,1,bravery at its finest,E:\datasets\MADE\3_graduation\parthplc\archive...
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,E:\datasets\MADE\3_graduation\parthplc\archive...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,80243,img/80243.png,1,mississippi wind chime,E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
495,83675,img/83675.png,0,i'm gonna be like phelps one day,E:\datasets\MADE\3_graduation\parthplc\archive...
496,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,E:\datasets\MADE\3_graduation\parthplc\archive...
497,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,E:\datasets\MADE\3_graduation\parthplc\archive...
498,9863,img/09863.png,0,diverse group of women,E:\datasets\MADE\3_graduation\parthplc\archive...


In [6]:
easyocr_reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [7]:
class ImageTransformType(IntEnum):
    No = 0
    Gray = 1
    Binary = 2
    BinaryInv = 3
    AdaptiveMean = 4
    AdaptiveMeanInv = 5
    AdaptiveGaussian = 6
    AdaptiveGaussianInv = 7

In [8]:
def recognize(path:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, 
              block_size:int, c:float) -> list:
    recognized_text = []    
    for i in range(len(path)):
        #print(path[i])
        image = cv2.imread(path[i], cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if image_transform == ImageTransformType.Gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        if image_transform == ImageTransformType.Binary:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY)
            
        if image_transform == ImageTransformType.BinaryInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY_INV)

        if image_transform == ImageTransformType.AdaptiveMean:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

        if image_transform == ImageTransformType.AdaptiveMeanInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussian:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussianInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if recognizer_name == 'easyocr':            
            recognized_text.append(' '.join(easyocr_reader.readtext(image, detail=0)))
            
        elif recognizer_name == 'pytesseract':
            recognized_text.append(pytesseract.image_to_string(image).replace('\n', ' '))
            
    return recognized_text


def loss(data:list, text:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, block_size:int, c:float):
    recognized_text = recognize(data, recognizer_name, image_transform, thresh, maxval, block_size, c)
    try:
        distances = []
        for i in range(len(data)):
            txt = text[i]
            distances.append(lev.distance(txt, recognized_text[i]) / len(text))

        distances_mean = np.mean(distances)
    except Exception as ex:
        return 100500
    return distances_mean

In [9]:
loss(test_data['path'].values, 
     test_data['text'].values,
     recognizer_name='easyocr',
     image_transform=ImageTransformType.No,
     thresh=0,
     maxval=0,
     block_size=0,
     c=0)

0.019604

In [10]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.No,
                                                                    ImageTransformType.Gray,
                                                                    ImageTransformType.Binary,
                                                                    ImageTransformType.BinaryInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='easyocr',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=0,
                c=0)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-11-08 18:01:32,267] A new study created in memory with name: no-name-d2740e01-b53c-4e80-a690-d76e09729d54
[I 2022-11-08 18:02:19,959] Trial 0 finished with value: 0.01946 and parameters: {'image_transform': <ImageTransformType.Gray: 1>, 'thresh': 209, 'maxval': 116}. Best is trial 0 with value: 0.01946.
[I 2022-11-08 18:03:08,283] Trial 1 finished with value: 0.019604 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 54, 'maxval': 126}. Best is trial 0 with value: 0.01946.
[I 2022-11-08 18:03:55,724] Trial 2 finished with value: 0.028956000000000003 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 144, 'maxval': 156}. Best is trial 0 with value: 0.01946.
[I 2022-11-08 18:04:44,739] Trial 3 finished with value: 0.019604 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 120, 'maxval': 143}. Best is trial 0 with value: 0.01946.
[I 2022-11-08 18:05:34,214] Trial 4 finished with value: 0.01946 and parame

[I 2022-11-08 18:31:48,156] Trial 37 finished with value: 0.018644 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 199, 'maxval': 51}. Best is trial 35 with value: 0.016232.
[I 2022-11-08 18:32:34,250] Trial 38 finished with value: 0.015792 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 240, 'maxval': 57}. Best is trial 38 with value: 0.015792.
[I 2022-11-08 18:33:20,945] Trial 39 finished with value: 0.023532000000000004 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 127, 'maxval': 58}. Best is trial 38 with value: 0.015792.
[I 2022-11-08 18:34:06,897] Trial 40 finished with value: 0.017668000000000003 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 221, 'maxval': 93}. Best is trial 38 with value: 0.015792.
[I 2022-11-08 18:34:53,056] Trial 41 finished with value: 0.016232 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 238, 'maxval': 

[I 2022-11-08 18:59:18,720] Trial 73 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 255, 'maxval': 77}. Best is trial 53 with value: 0.015359999999999999.
[I 2022-11-08 19:00:04,870] Trial 74 finished with value: 0.015744000000000005 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 250, 'maxval': 88}. Best is trial 53 with value: 0.015359999999999999.
[I 2022-11-08 19:00:51,465] Trial 75 finished with value: 0.016984000000000003 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 220, 'maxval': 66}. Best is trial 53 with value: 0.015359999999999999.
[I 2022-11-08 19:01:37,708] Trial 76 finished with value: 0.017724000000000004 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 228, 'maxval': 69}. Best is trial 53 with value: 0.015359999999999999.
[I 2022-11-08 19:02:23,712] Trial 77 finished with value: 0.016956000000000002 and parameters: {'i

[I 2022-11-08 19:25:36,188] Trial 108 finished with value: 0.016668 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 227, 'maxval': 49}. Best is trial 84 with value: 0.015284000000000002.
[I 2022-11-08 19:25:59,469] Trial 109 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 256, 'maxval': 31}. Best is trial 84 with value: 0.015284000000000002.
[I 2022-11-08 19:26:47,928] Trial 110 finished with value: 0.019604 and parameters: {'image_transform': <ImageTransformType.No: 0>, 'thresh': 140, 'maxval': 90}. Best is trial 84 with value: 0.015284000000000002.
[I 2022-11-08 19:27:34,205] Trial 111 finished with value: 0.015652000000000003 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 250, 'maxval': 51}. Best is trial 84 with value: 0.015284000000000002.
[I 2022-11-08 19:28:21,145] Trial 112 finished with value: 0.019304 and parameters: {'image_transform': <ImageTransformType.B

[I 2022-11-08 19:51:21,989] Trial 143 finished with value: 0.015868000000000004 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 239, 'maxval': 77}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 19:52:08,025] Trial 144 finished with value: 0.015320000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 247, 'maxval': 54}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 19:52:54,308] Trial 145 finished with value: 0.015552000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 60}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 19:53:40,680] Trial 146 finished with value: 0.015248000000000001 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 247, 'maxval': 53}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 19:54:25,939] Trial 147 finished with value: 0.015588000000000003 

[I 2022-11-08 20:16:50,550] Trial 178 finished with value: 0.10858 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 256, 'maxval': 52}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:17:36,159] Trial 179 finished with value: 0.01536 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 246, 'maxval': 73}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:18:22,093] Trial 180 finished with value: 0.0158 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 237, 'maxval': 72}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:19:07,944] Trial 181 finished with value: 0.015344000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 246, 'maxval': 66}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:19:54,140] Trial 182 finished with value: 0.015568000000000002 and parameters: {'image_transform': <Ima

[I 2022-11-08 20:43:24,322] Trial 213 finished with value: 0.015648000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 247, 'maxval': 45}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:44:12,830] Trial 214 finished with value: 0.020448 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 164, 'maxval': 54}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:45:00,967] Trial 215 finished with value: 0.015552000000000002 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 252, 'maxval': 60}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:45:48,118] Trial 216 finished with value: 0.015571999999999999 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 243, 'maxval': 50}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 20:46:11,353] Trial 217 finished with value: 0.10858 and parameters: {'image_t

[I 2022-11-08 21:09:28,455] Trial 248 finished with value: 0.015364000000000001 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 246, 'maxval': 72}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:10:14,544] Trial 249 finished with value: 0.015792 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 242, 'maxval': 77}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:11:00,785] Trial 250 finished with value: 0.015764 and parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 237, 'maxval': 70}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:11:46,409] Trial 251 finished with value: 0.015588000000000001 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 248, 'maxval': 75}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:12:32,398] Trial 252 finished with value: 0.015340000000000003 and parameters: {'ima

[I 2022-11-08 21:35:34,621] Trial 283 finished with value: 0.017308 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 230, 'maxval': 60}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:36:31,512] Trial 284 finished with value: 0.062456000000000005 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 46, 'maxval': 54}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:37:18,341] Trial 285 finished with value: 0.016132 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 241, 'maxval': 66}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:38:08,184] Trial 286 finished with value: 0.039892 and parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 97, 'maxval': 60}. Best is trial 118 with value: 0.015208000000000003.
[I 2022-11-08 21:38:54,561] Trial 287 finished with value: 0.015692000000000005 and parameters: {'image_tr

In [11]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.015208000000000003
Parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 62}


In [12]:
def objective(trial):
    #recognizer_name = trial.suggest_categorical("recognizer_name", ['easyocr', 'pytesseract'])
    image_transform = trial.suggest_categorical("image_transform", [ImageTransformType.AdaptiveMean,
                                                                   ImageTransformType.AdaptiveMeanInv,
                                                                   ImageTransformType.AdaptiveGaussian,
                                                                   ImageTransformType.AdaptiveGaussianInv])
    thresh = trial.suggest_int("thresh", 0, 256)
    maxval = trial.suggest_int("maxval", 0, 256)
    block_size = trial.suggest_int("block_size", 3, 21, 2)
    c = trial.suggest_int("c", -10, 10)
 
    return loss(test_data['path'].values, 
     test_data['text'].values,
                recognizer_name='easyocr',#recognizer_name,
                image_transform=image_transform,
                thresh=thresh,
                maxval=maxval,
                block_size=block_size,
                c=c)

study = optuna.create_study()
study.optimize(objective, n_trials=300)


[I 2022-11-08 21:47:47,566] A new study created in memory with name: no-name-4670eb10-9207-40fa-a9e8-85900ded4a14
[I 2022-11-08 21:48:36,048] Trial 0 finished with value: 0.051392 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 44, 'maxval': 145, 'block_size': 15, 'c': 10}. Best is trial 0 with value: 0.051392.
[I 2022-11-08 21:49:26,158] Trial 1 finished with value: 0.05351600000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 99, 'maxval': 243, 'block_size': 15, 'c': 0}. Best is trial 0 with value: 0.051392.
[I 2022-11-08 21:50:13,079] Trial 2 finished with value: 0.045224 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 78, 'maxval': 79, 'block_size': 9, 'c': 2}. Best is trial 2 with value: 0.045224.
[I 2022-11-08 21:51:04,696] Trial 3 finished with value: 0.05405200000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 121

[I 2022-11-08 22:13:45,006] Trial 31 finished with value: 0.032892000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 44, 'maxval': 121, 'block_size': 21, 'c': 7}. Best is trial 21 with value: 0.03184800000000001.
[I 2022-11-08 22:14:32,802] Trial 32 finished with value: 0.031892000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 80, 'maxval': 103, 'block_size': 21, 'c': 4}. Best is trial 21 with value: 0.03184800000000001.
[I 2022-11-08 22:15:21,278] Trial 33 finished with value: 0.033156000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 85, 'maxval': 99, 'block_size': 19, 'c': 3}. Best is trial 21 with value: 0.03184800000000001.
[I 2022-11-08 22:16:11,008] Trial 34 finished with value: 0.032544000000000003 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 110, 'maxval': 106, 'block_size': 21, 'c': 1}. Best is trial 

[I 2022-11-08 22:39:13,131] Trial 63 finished with value: 0.031568 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 158, 'maxval': 57, 'block_size': 19, 'c': 2}. Best is trial 61 with value: 0.030820000000000007.
[I 2022-11-08 22:40:00,053] Trial 64 finished with value: 0.031072000000000006 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 161, 'maxval': 60, 'block_size': 19, 'c': 4}. Best is trial 61 with value: 0.030820000000000007.
[I 2022-11-08 22:40:48,985] Trial 65 finished with value: 0.033992 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 174, 'maxval': 58, 'block_size': 17, 'c': 1}. Best is trial 61 with value: 0.030820000000000007.
[I 2022-11-08 22:41:35,457] Trial 66 finished with value: 0.032056 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 163, 'maxval': 33, 'block_size': 19, 'c': 8}. Best is trial 61 with value: 0.0308200000000000

[I 2022-11-08 23:04:10,686] Trial 94 finished with value: 0.030264000000000003 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 88, 'maxval': 38, 'block_size': 21, 'c': -2}. Best is trial 85 with value: 0.029484000000000003.
[I 2022-11-08 23:04:59,669] Trial 95 finished with value: 0.032600000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 87, 'maxval': 25, 'block_size': 21, 'c': -2}. Best is trial 85 with value: 0.029484000000000003.
[I 2022-11-08 23:05:47,846] Trial 96 finished with value: 0.0347 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 94, 'maxval': 40, 'block_size': 21, 'c': -3}. Best is trial 85 with value: 0.029484000000000003.
[I 2022-11-08 23:06:36,003] Trial 97 finished with value: 0.03188800000000001 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 71, 'maxval': 27, 'block_size': 21, 'c': -2}. Best is trial 85 with 

[I 2022-11-08 23:29:07,632] Trial 125 finished with value: 0.027716 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 130, 'maxval': 52, 'block_size': 21, 'c': -7}. Best is trial 116 with value: 0.027440000000000006.
[I 2022-11-08 23:29:54,855] Trial 126 finished with value: 0.028768000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 130, 'maxval': 52, 'block_size': 19, 'c': -7}. Best is trial 116 with value: 0.027440000000000006.
[I 2022-11-08 23:30:42,077] Trial 127 finished with value: 0.028144000000000006 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 146, 'maxval': 65, 'block_size': 21, 'c': -5}. Best is trial 116 with value: 0.027440000000000006.
[I 2022-11-08 23:31:28,936] Trial 128 finished with value: 0.027460000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 122, 'maxval': 55, 'block_size': 21, 'c': -8}. Best is tria

[I 2022-11-08 23:52:39,976] Trial 155 finished with value: 0.027452000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 177, 'maxval': 91, 'block_size': 21, 'c': -10}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-08 23:53:26,670] Trial 156 finished with value: 0.027236000000000003 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 160, 'maxval': 78, 'block_size': 21, 'c': -10}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-08 23:54:13,076] Trial 157 finished with value: 0.027200000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 160, 'maxval': 79, 'block_size': 21, 'c': -10}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-08 23:54:59,647] Trial 158 finished with value: 0.028652000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 161, 'maxval': 88, 'block_size': 19, 'c': -1

[I 2022-11-09 00:16:05,462] Trial 185 finished with value: 0.026904000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 210, 'maxval': 70, 'block_size': 21, 'c': -10}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-09 00:16:52,261] Trial 186 finished with value: 0.027244000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 223, 'maxval': 70, 'block_size': 21, 'c': -9}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-09 00:17:39,144] Trial 187 finished with value: 0.029908 and parameters: {'image_transform': <ImageTransformType.AdaptiveGaussian: 6>, 'thresh': 210, 'maxval': 59, 'block_size': 21, 'c': -10}. Best is trial 134 with value: 0.026884000000000005.
[I 2022-11-09 00:18:25,923] Trial 188 finished with value: 0.026904000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 202, 'maxval': 70, 'block_size': 21, 'c': -10}. Best 

[I 2022-11-09 00:39:42,735] Trial 215 finished with value: 0.027508 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 208, 'maxval': 56, 'block_size': 21, 'c': -9}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 00:40:32,957] Trial 216 finished with value: 0.037048000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMeanInv: 5>, 'thresh': 216, 'maxval': 58, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 00:41:20,981] Trial 217 finished with value: 0.027044000000000006 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 205, 'maxval': 63, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 00:42:08,671] Trial 218 finished with value: 0.026832 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 188, 'maxval': 56, 'block_size': 21, 'c': -10}. Best is trial 214 

[I 2022-11-09 01:03:17,241] Trial 245 finished with value: 0.026900000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 214, 'maxval': 65, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:04:04,501] Trial 246 finished with value: 0.027388000000000003 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 216, 'maxval': 57, 'block_size': 21, 'c': -9}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:04:51,867] Trial 247 finished with value: 0.026900000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 232, 'maxval': 65, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:05:38,834] Trial 248 finished with value: 0.026900000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 243, 'maxval': 65, 'block_size': 21, 'c': -10

[I 2022-11-09 01:26:54,595] Trial 275 finished with value: 0.026884000000000005 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 243, 'maxval': 66, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:27:41,626] Trial 276 finished with value: 0.027480000000000004 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 245, 'maxval': 44, 'block_size': 21, 'c': -10}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:28:28,668] Trial 277 finished with value: 0.027360000000000002 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 256, 'maxval': 67, 'block_size': 21, 'c': -9}. Best is trial 214 with value: 0.026816000000000003.
[I 2022-11-09 01:29:15,300] Trial 278 finished with value: 0.026832 and parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 237, 'maxval': 56, 'block_size': 21, 'c': -10}. Best is t

In [13]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

Value: 0.026816000000000003
Parameters: {'image_transform': <ImageTransformType.AdaptiveMean: 4>, 'thresh': 206, 'maxval': 57, 'block_size': 21, 'c': -10}
